# 01B: Make My Own Code

This notebook shows how to build a `QldpcCode` from parity-check matrices (`hz`, `hx`).

For examples, we use small quantum Tanner code files from:
https://aleverrier.github.io/qtanner-search/best_codes/index.html

### Important Note

The helper function `load_mtx_binary(...)` is needed only because QTanner examples are distributed as Matrix Market (`.mtx`) files.

In principle, QUITS only needs binary NumPy arrays. If you already have `hz` and `hx` as arrays of 0/1, you can skip file loading entirely.

In [1]:
from pathlib import Path

import numpy as np
from scipy.io import mmread

from quits.qldpc_code import QldpcCode

In [2]:
# Use this only when reading parity-check matrices from `.mtx` files.
def load_mtx_binary(path: str | Path, as_dense: bool = True) -> np.ndarray:
    """Load one Matrix Market parity-check file as a binary NumPy array."""
    M = mmread(str(path))
    if as_dense and hasattr(M, "toarray"):
        M = M.toarray()

    M = np.asarray(M, dtype=np.int8) % 2
    if not np.isin(M, [0, 1]).all():
        raise ValueError(f"{path} is not binary after mod-2 conversion.")
    return M

In [3]:
# If running from doc/, this points to the repo's parity_check_matrices folder.
folder = Path("../parity_check_matrices")

# Set file names explicitly (replace with your actual QTanner-exported files).
hz_file = "633__C2xC2_AAp0_0_0_0_1_2_3_BBp0_0_0_1_1_2_2_k12_d11__Hz.mtx"
hx_file = "633__C2xC2_AAp0_0_0_0_1_2_3_BBp0_0_0_1_1_2_2_k12_d11__Hx.mtx"

hz = load_mtx_binary(folder / hz_file)
hx = load_mtx_binary(folder / hx_file)

### Load QldpcCode object

In [4]:
code = QldpcCode.from_parity_checks(hz, hx, compute_logicals=True)

print("  hz shape:", code.hz.shape)
print("  hx shape:", code.hx.shape)
print("  lz shape:", code.lz.shape)
print("  lx shape:", code.lx.shape)
print("\nCSS report:")
code.verify_css_logicals()

  hz shape: (72, 144)
  hx shape: (72, 144)
  lz shape: (12, 144)
  lx shape: (12, 144)

CSS report:


{'css_condition': True,
 'lz_commutes_with_X': True,
 'lx_commutes_with_Z': True,
 'rank_hz': 66,
 'rank_hx': 66,
 'rank_lz': 12,
 'rank_lx': 12,
 'k_expected': 12,
 'lz_independent_mod_Z_stabilizers': True,
 'lx_independent_mod_X_stabilizers': True,
 'rank_hz_plus_lz': 78,
 'rank_hx_plus_lx': 78,
 'dim_ker_hz': 78,
 'dim_ker_hx': 78,
 'hx_plus_lx_spans_ker_hz': True,
 'hz_plus_lz_spans_ker_hx': True,
 'same_logicals_ZX_anticommute': True,
 'different_logicals_ZX_commute': True,
 'all_tests_passed': True}

### Build a `zxcoloration` Circuit

Now that `code` is created from `hz` and `hx`, you can build a circuit using the `zxcoloration` strategy.

For a full walkthrough of the ZX-coloration construction flow, see:
`doc/02C_zxcoloration_circuit_generation.ipynb`

In [6]:
from quits import CircuitBuildOptions, ErrorModel

# Example circuit-generation parameters
p = 1e-3
num_rounds = 2

error_model = ErrorModel(
    idle_error=p,
    sqgate_error=p,
    tqgate_error=p,
    spam_error=p,
)

circuit_build_options = CircuitBuildOptions(
    get_all_detectors=False,
    noisy_zeroth_round=True,
    noisy_final_meas=False,
)

circuit = code.build_circuit(
    strategy="zxcoloration",
    error_model=error_model,
    num_rounds=num_rounds,
    basis="Z",
    circuit_build_options=circuit_build_options,
)

print("Built zxcoloration circuit")
print("Depth:", getattr(code, "depth", None))
print("See doc/02C_zxcoloration_circuit_generation.ipynb for full details.")
print(circuit)

Built zxcoloration circuit
Depth: 18
See doc/02C_zxcoloration_circuit_generation.ipynb for full details.
R 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
X_ERROR(0.001) 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 13